In [1]:
# 텐서플로의 그레이디언트 테이프
import tensorflow as tf
x = tf.Variable(0.)

In [3]:
with tf.GradientTape() as tape:
    y = 2*x+3
grad_of_y_wrt_x = tape.gradient(x,y)    

In [5]:
x = tf.Variable(tf.zeros(2,2))
with tf.GradientTape() as tape:
    y = 2*x+3
grad_of_y_wrt_x = tape.gradient(x,y)    

In [7]:
W = tf.Variable(tf.random.uniform((2,2))) # 가중치를 초기화
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2,2))
with tf.GradientTape() as tape:
    y = tf.matmul(x,W) + b
grad_of_y_wrt_W_and_b = tape.gradient(y, [W,b])    

In [12]:
# 데이터 로드
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test)  = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)

In [13]:
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

In [15]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['acc'])

In [16]:
model.fit(x_train,y_train,epochs=5,batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.2692 - acc: 0.9236
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1111 - acc: 0.9672
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0724 - acc: 0.9789
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0517 - acc: 0.9849
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0388 - acc: 0.9885


In [17]:
# 텐서플로우를 이용해서 위의 식을 구현해 보기

In [27]:
class NativeDense:
    def __init__(self,input_size, output_size, activation):
        self.activation = activation
        w_shape = (input_size,output_size)
        w_initial_value = tf.random.uniform(w_shape,minval=0, maxval=1e-1)  # 0.1
        self.W = tf.Variable(w_initial_value)
        
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]

In [28]:
class NativeSequential:
    def __init__(self, layers):
        self.layers = layers
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [29]:
model = NativeSequential([
    NativeDense(input_size=28*28,output_size=512,activation=tf.nn.relu),
    NativeDense(input_size=512,output_size=10,activation=tf.nn.relu)    
])
assert len(model.weights) == 4

In [32]:
#배치 제너레이터
import math
class BachGenerator:
    def __init__(self, images,labels,batch_size=128):
        assert len(images) == len(label)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)  
    def next(self):
        images = self.images[self.index : self.index+self.batch_size]
        labes = self.labels[self.index : self.index+self.batch_size]
        selft.index += self.batch_size
        return images, labels

In [30]:
# 훈련스텝 실행
def one_training_step(model, image_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(image_batch)
        per_sample_losses =  tf.keras.losses.sparse_categorical_crossentropy(labels_batch,predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss,model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [33]:
# 가중치 업데이트
learning_rate = 1e-3 # 0.001
def update_weights(gradients, weights):
    for g, w in  zip(gradients, weights):
        w.assign_sub(g*learning_rate)

In [34]:
# 옵티마이져
from tensorflow.keras import optimizers
optimizer = optimizers.SGD(learning_rate=1e-3)
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [35]:
# 훈련
def fit(model,images,labels,epochs,batch_size=128):
    for epoch_counter in range(epochs):
        print(f"에포크 {epoch_counter}")
        batch_generator = BachGenerator(images,labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch =  batch_generator.next()
            loss = one_training_step(model, image_batch, labels_batch)
            if batch_counter % 100 ==0:
                print(f"{batch_counter}번째 배치 손실 : {loss:.2f}")